In [ ]:
import csv
import numpy as np
import pandas as pd

header = []
ids = []
data = []
with open("data.csv", "r") as f:
    reader = csv.reader(f)
    header = next(reader)[1:]
    for row in reader:
        name = row[0]
        ids.append(name)
        data.append(list(map(float, row[1:])))

df = pd.DataFrame(data, columns=header)
df['name'] = ids
df = df.set_index('name')
df

All the criteria are gain criteria except **weight** and **price**

In [ ]:
GAIN = 0
COST = 1
criteria_types = [GAIN, GAIN, GAIN, GAIN, GAIN, COST, COST]

# Preferencial information

List of pairwise comparisons prepared in the previous project, enriched with additional pairwise comparisons. We have added two cycles to introduce some inconsistency into data.

- \> means preference
- ~ means indifference

In [ ]:
preferences = [
    ('Samsung Galaxy S22 Ultra', '>', 'Pixel 7'),
    ('Samsung Galaxy S22 Ultra' , '>', 'Samsung S24+'),
    ('Pixel 7', '>', 'Pixel 8'),
    ('Samsung S24 Ultra', '~', 'iPhone 15 Pro Max'),
    ('Samsung S23 FE', '>', 'iPhone 15 Pro'),
    ('Vivo X80 Pro', '~', 'Samsung S23 FE'),
    ('Pixel 8 Pro', '>', 'Pixel 8'),
    ('Oneplus 11 5G', '~', 'Vivo X80 Pro'),
    ('Samsung S24', '~', 'Samsung S23 FE'),
    ('Samsung S24+', '~', 'iPhone 15 Pro Max'),
    ('iPhone 15', '>', 'Samsung S24+'),
    ('Xiaomi 12 Pro', '>', 'Vivo X80 Pro'),
    ('iPhone 15', '>', 'iPhone SE the 3rd'),
    ('iPhone 15', '>', 'iPhone 15 Pro'), # inconsistency A > B and B > C and C > A 
    ('iPhone 15 Pro', '>', 'iPhone 15 Pro Max'),
    ('iPhone 15 Pro Max', '>', 'iPhone 15'),
    ('iPhone SE the 3rd', '>', 'Samsung S24+'), # inconsistency A > B and B > A
    ('Samsung S24+', '>', 'iPhone SE the 3rd')
]

Preparing the linear program

In [ ]:
import pulp
from itertools import pairwise

prob = pulp.LpProblem("Resolve Inconsistencies", pulp.LpMinimize)

MAX_WEIGHT = 0.5
MIN_WEIGHT = 0.1


def add_constraints(prob, u_vars):
    best_u = []
    for i, (c, ctype) in enumerate(zip(df, criteria_types)):
        series = sorted(df[c], reverse=bool(ctype))
        
        # Worst values for certain criteria should be zero
        worst = series[0]
        worstid = f'u_{i}_{worst}'
        if worstid not in u_vars:
            u_vars[worstid] = pulp.LpVariable(worstid, lowBound=0, upBound=0)

        # Best values for certain criteria should be at most MAX_WEIGHT and sum to 1
        best = series[-1]
        bestid = f'u_{i}_{best}'
        if bestid not in u_vars:
            u_vars[bestid] = pulp.LpVariable(bestid, lowBound=MIN_WEIGHT, upBound=MAX_WEIGHT)
        best_u.append(u_vars[bestid])

        # Monotonicity constraints
        for a, b in pairwise(series):
            aid = f'u_{i}_{a}'
            bid = f'u_{i}_{b}'
            if aid not in u_vars:
                u_vars[aid] = pulp.LpVariable(aid, lowBound=0, upBound=MAX_WEIGHT)
            if bid not in u_vars:
                u_vars[bid] = pulp.LpVariable(bid, lowBound=0, upBound=MAX_WEIGHT)
            prob += u_vars[aid] <= u_vars[bid]

    # Normalization constraint of best values in each criteria
    constraint = pulp.LpConstraint(pulp.lpSum(best_u), sense=pulp.LpConstraintEQ, rhs=1)
    prob += constraint


u_vars = {}
add_constraints(prob, u_vars)

Non negativity constraints for u already included by setting lowBound to 0

# 2.1 Resolving inconsistencies

In [ ]:
binary_variables = []

EPSILON = 1e-6

for a, ctype, b in preferences:
    v_ab = pulp.LpVariable(f'v_{a},{b}', cat=pulp.LpBinary)
    binary_variables.append(v_ab)
    A = []
    for i, v in enumerate(df.loc[a]):
        identifier = f"u_{i}_{v}"
        if identifier not in u_vars:
            u_vars[identifier] = pulp.LpVariable(identifier, lowBound=0, upBound=MAX_WEIGHT)
        A.append(u_vars[identifier])

    B = []
    for i, v in enumerate(df.loc[b]):
        identifier = f"u_{i}_{v}"
        if identifier not in u_vars:
            u_vars[identifier] = pulp.LpVariable(identifier, lowBound=0, upBound=MAX_WEIGHT)
        B.append(u_vars[identifier])

    if ctype == '>':
        prob += pulp.lpSum(A) >= pulp.lpSum(B) + EPSILON - v_ab
    else:
        prob += pulp.lpSum(A) >= pulp.lpSum(B) - v_ab
        prob += pulp.lpSum(B) >= pulp.lpSum(A) - v_ab

prob += pulp.lpSum(binary_variables)

In [ ]:
prob.solve(pulp.PULP_CBC_CMD(msg=0))

## Inconsistent comparisons to remove

In [ ]:
consistent_preferences = []
for i, v in enumerate(binary_variables):
    if v.value() == 1:
        print(*preferences[i])
    else:
        consistent_preferences.append(preferences[i])

In [ ]:
import matplotlib.pyplot as plt

def plot_uta(u_vars):
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
    axs = axs.flatten()

    for i, c in enumerate(df.columns):
        axs[i].set_title(c)
        minimum = df[c].min()
        maximum = df[c].max()
        axs[i].set_xlim(minimum, maximum)
        axs[i].set_xticks(np.round(np.linspace(minimum, maximum, 6), 2))
        axs[i].set_ylim(0, MAX_WEIGHT)
        axs[i].set_ylabel(f'u_{i}')
        axs[i].set_xlabel(f'g_{i}')


        x = sorted(df[c])
        y = [u_vars[f'u_{i}_{v}'].value() for v in x]
        axs[i].plot(x, y)

    axs[7].axis('off')

    plt.show()

plot_uta(u_vars)

## Finding all other subsets of comparisons to remove to achieve consistency

In [ ]:
count = 0
while True:
    ones = list(filter(lambda v: v.value() == 1, binary_variables))
    new_constraint = pulp.lpSum(ones) <= len(ones) - 1 
    prob += new_constraint
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    if prob.sol_status != 1:
        break


    print('Criteria to remove:')
    for i, v in enumerate(binary_variables):
        if v.value() == 1:
            print('-', *preferences[i])
    plot_uta(u_vars)
    count += 1

In [ ]:
print(count + 1)

## Discuss the results
As you can see there are 10 ways to remove comparisions to make the data consistent. Algorithm works as expected, weights of criteria do sum to 1 and non of the weights is smaller than the threshold we have set at the beggining (0.1).

# 2.2 - Minimize sum of over and under estimation errors

In [ ]:
prob = pulp.LpProblem("Minimize sum of over and under estimation errors", pulp.LpMinimize)

u_vars = {}
add_constraints(prob, u_vars)

under_estimation_vars = {}
over_estimation_vars = {}
EPSILON = 1e-6

for a, ctype, b in consistent_preferences:
    for x in [a,b]:
        x_str = str(x)
        if x_str not in under_estimation_vars:
            under_estimation_vars[x_str] = pulp.LpVariable(f'under_{x_str}', lowBound=0)
        if x not in over_estimation_vars:
            over_estimation_vars[x_str] = pulp.LpVariable(f'over_{x_str}', lowBound=0)

    A = []
    for i, v in enumerate(df.loc[a]):
        identifier = f"u_{i}_{v}"
        if identifier not in u_vars:
            u_vars[identifier] = pulp.LpVariable(identifier, lowBound=0, upBound=MAX_WEIGHT)
        A.append(u_vars[identifier])

    B = []
    for i, v in enumerate(df.loc[b]):
        identifier = f"u_{i}_{v}"
        if identifier not in u_vars:
            u_vars[identifier] = pulp.LpVariable(identifier, lowBound=0, upBound=MAX_WEIGHT)
        B.append(u_vars[identifier])

    sumA = pulp.lpSum(A) - over_estimation_vars[str(a)] + under_estimation_vars[str(a)]
    sumB = pulp.lpSum(B) - over_estimation_vars[str(b)] + under_estimation_vars[str(b)]

    if ctype == '>':
        prob += sumA >= sumB + EPSILON
    else:
        prob += pulp.LpConstraint(sumA - sumB, sense=pulp.LpConstraintEQ , rhs=0)

prob += pulp.lpSum(under_estimation_vars) + pulp.lpSum(over_estimation_vars)
prob.solve()

In [ ]:
plot_uta(u_vars=u_vars)

Value of the objective function

In [ ]:
prob.objective.value()

## Discuss the results